# **Dissipative plane in a unit cube**

Test the elementary load cases - tension, compression and sliding

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
from ibvpy.bcond import BCSlice
from ibvpy.fets import FETS3D8H
from ibvpy.tmodel.mats3D import MATS3DElastic
from bmcs_matmod.ms1.ms1 import MS13D
import bmcs_matmod.ms1.concrete_material_db as mp_db
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from ibvpy.sim.tstep_bc import TStepBC
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid

import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
from bmcs_matmod.slide.vslide_34_ij import MATS3DSlideStrain, Slide34

In [ ]:
from bmcs_matmod.slide.vslide_34_ij import MATS3DSlideStrain
class Debugging(MATS3DSlideStrain):
    def get_corr_pred(self, eps_Emab_n1, tn1, **state):
        r'''
        Corrector predictor computation.
        '''
        n_i = self.n_a
        eps_ij = eps_Emab_n1
        eps_N = np.einsum('...ij,...i,...j->...', eps_ij, n_i, n_i)
        #
        # Change #1
        # Instead of concatenation I reuse the eps_NT_Ema - in-place assuming that the first eps_NT_Ema[...,0] is zero 
        # Check if it is true
        #
        # eps_T = self.get_eps_T(eps_ij, n_i)
        # eps_T = np.sqrt(np.einsum('...i,...i->...',eps_T, eps_T))
        eps_NT_Ema = self.get_eps_T(eps_ij, n_i)
        eps_NT_Ema[..., 0] = eps_N
        # print(eps_N.shape, eps_T)
        # eps_NT_Ema = np.concatenate([np.transpose(eps_N), np.transpose(eps_T)], axis=-1)
        # print('eps_NT_Ema', eps_NT_Ema.shape)
        # print(self.state_var_shapes)
        se = self.slide_displ_
        sig_NT_Ema, D_Emab = se.get_corr_pred(eps_NT_Ema, tn1, **state)
        eps_N_p, eps_T_p_x, eps_T_p_y = state['w_pi'], state['s_pi_x'], state['s_pi_y']
        eps_T = self.get_eps_T(eps_ij, n_i)
        eps_T_p_i = self.get_eps_T_p(eps_T_p_x, eps_T)
        omega_N_Em, omega_T_Em = state['omega_N'], state['omega_T']
        phi_Emab = np.zeros_like(eps_Emab_n1)
        phi_Emab[..., 0, 0] = np.sqrt(1 - omega_N_Em)
        phi_Emab[..., 1, 1] = np.sqrt(1 - omega_T_Em)
        phi_Emab[..., 2, 2] = np.sqrt(1 - omega_T_Em)
        beta_Emijkl = np.einsum('...ik,...lj->...ijkl', phi_Emab, phi_Emab)
        eps_ij_p = (np.einsum('i,...j->...ij', n_i, eps_T_p_i) +
                    np.einsum('...i,j->...ij', eps_T_p_i, n_i) +
                    np.einsum('...,i,j->...ij', eps_N_p, n_i, n_i)
                   )
        D_abef = self.D_abef
        D_Emabcd = np.einsum('...ijkl,klrs,...rstu->...ijtu', beta_Emijkl, D_abef, beta_Emijkl)
        sigma_Emab = np.einsum('...ijkl,...kl->...ij', D_Emabcd, (eps_Emab_n1 - eps_ij_p))
        return sigma_Emab, D_Emabcd

In [ ]:
size = 1
n_e = 1
xmodel = XDomainFEGrid(coord_min=(0, 0, 0),
                       coord_max=(size ,size, size),
                       shape=(n_e, n_e, n_e),
                     integ_factor=1,
                     fets=FETS3D8H())
E = 34000
u_N_f = 0.1
u_T_f = 0.6
S_N = 1/(E * u_N_f**2)
S_T = 1/(E * u_T_f**2)
material_params =  dict(
     gamma_T=5, K_T=0, S_T=100, c_T=6, bartau=3, 
     S_N=100, c_N = 3, m = 0.05, f_t=3, f_c=60, f_c0 = 40, eta=0.)
bond_m = Slide34(**material_params)
m_inelastic = Debugging(slide_displ=bond_m, E=34000, nu=0.2, n_a=[1, 0, 0])
m_elastic = MATS3DElastic(E=34000, nu=0.2)
m = TStepBC(
    domains=[(xmodel, m_inelastic),
             ]
)

In [ ]:
w_max = 0.01 * size
#w_max = -1
bc_l_x = BCSlice(slice=xmodel.mesh[0, :, :, 0, :, :],
                  var='u', dims=[0], value=0.0)
bc_r_x = BCSlice(slice=xmodel.mesh[-1, :, :, -1, :, :],
                 var='u', dims=[1], value=w_max)
bc_xy_z0 = BCSlice(slice=xmodel.mesh[:, :, 0, :, :, 0],
                  var='u', dims=[2], value=0)
bc_xz_y0 = BCSlice(slice=xmodel.mesh[:, 0, :, :, 0, :],
                 var='u', dims=[1], value=0)

In [ ]:
bc1 = [bc_l_x, bc_r_x, bc_xy_z0, bc_xz_y0]

In [ ]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
# Accessing the following property - ensures serialization
s.tstep.fe_domain.serialized_subdomains
s.reset()
s.run()

In [ ]:
U_last = m.hist.U_t[-1]
F_last = m.hist.U_t[-1]
U_t = np.average(m.hist.U_t[:, bc_r_x.dofs], axis=-1)
F_t = np.sum(m.hist.F_t[:, bc_r_x.dofs], axis=-1)
fig, ax = plt.subplots(1, 1)
ax.plot(U_t, F_t)

In [ ]:
if False:
    from mayavi import mlab
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()